## Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


__What variable(s) are the target(s) for your model?__

'IS_SUCCESSFUL'

__What variable(s) are the feature(s) for your model?__

All other columns

In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [40]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [41]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df['APPLICATION_TYPE'].value_counts()
app_type

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [43]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type.loc[app_type.values < 400].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [44]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [45]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts.loc[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [46]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_counts.loc[class_counts.values < 1500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [60]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_hot = list(application_df.select_dtypes(include="object").columns)
encoded_columns = pd.get_dummies(application_df[categorical_hot]).astype(int)
encoded_columns.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [48]:
# Split our preprocessed data into our features and target arrays

# Isolate the target array
y = application_df['IS_SUCCESSFUL']

# Isolate the feature array
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Replace features with encoded equivalents
X.drop(categorical_hot, axis=1, inplace=True)

# Calculate updated feature array
X = pd.concat([X, encoded_columns], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [51]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [53]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# Initialise sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer1,
    input_dim = number_input_features,
    activation = "relu"
))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer2,
    activation = "relu"
))


# Output layer
nn.add(tf.keras.layers.Dense(
    units = 1,
    activation = "sigmoid"
))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Compile the model
nn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [55]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoints filepath
filepath = "weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"

# Create a checkpoint
checkpoint = ModelCheckpoint(
    filepath,
    monitor='accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# Create the callbacks list
callbacks_list = [checkpoint]

In [58]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs = 200,
    callbacks = callbacks_list,
    verbose = 0
)


Epoch 1: accuracy did not improve from 0.73912

Epoch 2: accuracy did not improve from 0.73912

Epoch 3: accuracy did not improve from 0.73912

Epoch 4: accuracy did not improve from 0.73912

Epoch 5: accuracy did not improve from 0.73912

Epoch 6: accuracy did not improve from 0.73912

Epoch 7: accuracy did not improve from 0.73912

Epoch 8: accuracy did not improve from 0.73912

Epoch 9: accuracy did not improve from 0.73912

Epoch 10: accuracy did not improve from 0.73912

Epoch 11: accuracy did not improve from 0.73912

Epoch 12: accuracy did not improve from 0.73912

Epoch 13: accuracy did not improve from 0.73912

Epoch 14: accuracy did not improve from 0.73912

Epoch 15: accuracy did not improve from 0.73912

Epoch 16: accuracy did not improve from 0.73912

Epoch 17: accuracy did not improve from 0.73912

Epoch 18: accuracy did not improve from 0.73912

Epoch 19: accuracy did not improve from 0.73912

Epoch 20: accuracy did not improve from 0.73912

Epoch 21: accuracy did not i

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8169 - accuracy: 0.7314 - 87ms/epoch - 323us/step
Loss: 0.8168618083000183, Accuracy: 0.7314285635948181


In [31]:
# Export our model to HDF5 file
nn.save('iteration_1.h5', save_format='h5')

In [61]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    # Choose activation function in hidden layers
    activation_layer1 = hp.Choice('activation', ['relu', 'leaky_relu', 'tanh'])
    activation_layer2 = hp.Choice('activation', ['relu', 'leaky_relu', 'tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(
        units = hp.Int(
            'first_units',
            min_value = 1,
            max_value = number_input_features * 2 - 1,
            step = 10),
        activation = activation_layer1,
        input_dim = number_input_features
    ))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(
            units = hp.Int(
                'units_' + str(i),
                min_value = 1,
                max_value = number_input_features * 2 - 1,
                step = 10),
            activation = activation_layer2
        ))
    
    # Add the output layer
    nn_model.add(tf.keras.layers.Dense(
        units = 1,
        activation = "sigmoid"
    ))
    
    # Compile the model
    nn_model.compile(
        loss = "binary_crossentropy",
        optimizer = "adam",
        metrics = ["accuracy"]
    )
    
    return(nn_model)

In [62]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2
)

In [63]:
tuner.search(
    X_train_scaled,
    y_train,
    epochs = 20,
    validation_data = (X_test_scaled, y_test)
)

Trial 60 Complete [00h 00m 11s]
val_accuracy: 0.7344606518745422

Best val_accuracy So Far: 0.7371428608894348
Total elapsed time: 00h 04m 58s


In [64]:
# Get the top 3 model hyperparameters
best_hyper = tuner.get_best_hyperparameters(3)

from pprint import pprint
for model in best_hyper:
    pprint(model.values)

{'activation': 'relu',
 'first_units': 11,
 'num_layers': 5,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0044',
 'units_0': 71,
 'units_1': 41,
 'units_2': 81,
 'units_3': 81,
 'units_4': 31,
 'units_5': 81}
{'activation': 'leaky_relu',
 'first_units': 41,
 'num_layers': 5,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0013',
 'units_0': 81,
 'units_1': 61,
 'units_2': 21,
 'units_3': 71,
 'units_4': 71,
 'units_5': 71}
{'activation': 'tanh',
 'first_units': 21,
 'num_layers': 4,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0049',
 'units_0': 71,
 'units_1': 1,
 'units_2': 11,
 'units_3': 61,
 'units_4': 41,
 'units_5': 71}
